In [1]:
import math

def butterworth_biquad(fs, fc, filter_type):
    omega = 2 * math.pi * fc / fs
    sn = math.sin(omega)
    cs = math.cos(omega)
    alpha = sn / math.sqrt(2)  # Q = 1/sqrt(2) for Butterworth

    if filter_type == 'low':
        b0 = (1 - cs) / 2
        b1 = 1 - cs
        b2 = (1 - cs) / 2
    elif filter_type == 'high':
        b0 = (1 + cs) / 2
        b1 = -(1 + cs)
        b2 = (1 + cs) / 2
    elif filter_type == 'allpass':
        b0 = 1 - alpha
        b1 = -2 * cs
        b2 = 1 + alpha
    else:
        raise ValueError("filter_type must be 'low', 'high', or 'allpass'")

    a0 = 1 + alpha
    a1 = -2 * cs
    a2 = 1 - alpha

    b0 /= a0
    b1 /= a0
    b2 /= a0
    a1 /= a0
    a2 /= a0

    return b0, b1, b2, a1, a2

class Biquad:
    def __init__(self):
        self.state1 = 0.0
        self.state2 = 0.0

    def process(self, x, b0, b1, b2, a1, a2):
        y = b0 * x + self.state1
        self.state1 = b1 * x - a1 * y + self.state2
        self.state2 = b2 * x - a2 * y
        return y
    def process_allpass(self, x, b0, b1, b2, a1, a2):
        y = b0 * x + self.state1
        self.state1 = b1 * x - a1 * y + self.state2
        self.state2 = b2 * x - a2 * y
        return y

fs = 96000
low_cut = 125 # 83.5
high_cut = 2500

low_biquad1 = Biquad()
low_biquad2 = Biquad()
b0, b1, b2, a1, a2 = butterworth_biquad(fs, low_cut, 'low')

high_biquad1 = Biquad()
high_biquad2 = Biquad()
hb0, hb1, hb2, ha1, ha2 = butterworth_biquad(fs, high_cut, 'high')

mid_biquad1 = Biquad()
mid_biquad2 = Biquad()
mb0, mb1, mb2, ma1, ma2 = butterworth_biquad(fs, low_cut, 'allpass')
m2b0, m2b1, m2b2, m2a1, m2a2 = butterworth_biquad(fs, high_cut, 'allpass')

# print("lowpass coeffs", b0, b1, b2, a1, a2)
# print("allpass1 coeffs", mb0, mb1, mb2, ma1, ma2)
# print("allpass2 coeffs", m2b0, m2b1, m2b2, m2a1, m2a2)
# print("highpass coeffs", hb0, hb1, hb2, ha1, ha2)

print(f"// computed by testlrsplitter.ipynb for {low_cut}hz / {high_cut}hz cutoffs at {fs}hz")
print(f"static const float lowpass_b0 = {b0}f; // and b2")
print(f"static const float lowpass_b1 = {b1}f;")
print(f"static const float lowpass_a1 = {a1}f;")
print(f"static const float lowpass_a2 = {a2}f;")
print()
print(f"static const float highpass_b0 = {hb0}f; // and b2")
print(f"static const float highpass_b1 = {hb1}f;")
print(f"static const float highpass_a1 = {ha1}f;")
print(f"static const float highpass_a2 = {ha2}f;")



// computed by testlrsplitter.ipynb for 125hz / 2500hz cutoffs at 96000hz
static const float lowpass_b0 = 1.6636798430524727e-05f; // and b2
static const float lowpass_b1 = 3.3273596861049454e-05f;
static const float lowpass_a1 = -1.9884301203029076f;
static const float lowpass_a2 = 0.9884966674966296f;

static const float highpass_b0 = 0.890724065398853f; // and b2
static const float highpass_b1 = -1.781448130797706f;
static const float highpass_a1 = -1.7694710382281469f;
static const float highpass_a2 = 0.793425223367265f;


In [18]:
import numpy as np
# computed for 83.5hz / 2500hz cutoffs
lowpass_b0 = 6.642894505293559e-05  # and b2
lowpass_b1 = 0.00013285789010587118
lowpass_a1 = -1.9768147255510047
lowpass_a2 = 0.9770804413312165

highpass_b0 = 0.7057233874067733  # and b2
highpass_b1 = -1.4114467748135466
highpass_a1 = -1.3228873574952644
highpass_a2 = 0.5000061921318292

allpass_lo_b0 = lowpass_a2
allpass_lo_b1 = lowpass_a1
allpass_hi_b0 = highpass_a2
allpass_hi_b1 = highpass_a1

# b0 and b2 are the same for low and high pass...
def process_biquad_low_or_high(x, state, b0and2, b1, a1, a2):
    xb02 = x * b0and2
    y = xb02 + state[0]
    state[0] = b1 * x - a1 * y + state[1]
    state[1] = xb02 - a2 * y
    return y

# b2 is 1, b1 and a1 are the same, b0 is the same as a2
def process_allpass(x, state, b0anda2, b1anda1):
    y = b0anda2 * x + state[0]
    state[0] = b1anda1 * (x - y) + state[1]
    state[1] = x - b0anda2 * y
    return y


biquad_state = np.zeros(24)

def splitter(l, r):
    l_low = process_biquad_low_or_high(l, biquad_state[0:2], lowpass_b0, lowpass_b1, lowpass_a1, lowpass_a2)
    r_low = process_biquad_low_or_high(r, biquad_state[2:4], lowpass_b0, lowpass_b1, lowpass_a1, lowpass_a2)
    l_low = process_biquad_low_or_high(l_low, biquad_state[4:6], lowpass_b0, lowpass_b1, lowpass_a1, lowpass_a2)
    r_low = process_biquad_low_or_high(r_low, biquad_state[6:8], lowpass_b0, lowpass_b1, lowpass_a1, lowpass_a2)
    
    l_high = process_biquad_low_or_high(l, biquad_state[8:10], highpass_b0, highpass_b1, highpass_a1, highpass_a2)
    r_high = process_biquad_low_or_high(r, biquad_state[10:12], highpass_b0, highpass_b1, highpass_a1, highpass_a2)
    l_high = process_biquad_low_or_high(l_high, biquad_state[12:14], highpass_b0, highpass_b1, highpass_a1, highpass_a2)
    r_high = process_biquad_low_or_high(r_high, biquad_state[14:16], highpass_b0, highpass_b1, highpass_a1, highpass_a2)
    
    l_allpass = process_allpass(l, biquad_state[16:18], allpass_lo_b0, allpass_lo_b1)
    r_allpass = process_allpass(r, biquad_state[18:20], allpass_lo_b0, allpass_lo_b1)
    l_allpass = process_allpass(l_allpass, biquad_state[20:22], allpass_hi_b0, allpass_hi_b1)
    r_allpass = process_allpass(r_allpass, biquad_state[22:24], allpass_hi_b0, allpass_hi_b1)
    
    l_mid = l_allpass- l_low - l_high
    r_mid = r_allpass- r_low - r_high
    
    return (l_low, r_low, l_mid, r_mid, l_high, r_high)

from scipy.io import wavfile
fs=32000
duration = 10  # seconds
f0 = 50  # start frequency
f1 = 5000  # end frequency
t = np.linspace(0, 1, int(fs * duration))
k = np.exp(np.log(f1/f0) * t + np.log(f0))
phase = 2 * np.pi * np.cumsum(k) / fs
sweep = np.sin(phase)*0.5
wavfile.write('sweep.wav', fs, np.int16(sweep*32767))
lo,mid,hi=[],[],[]
for x in sweep:
    l_low, r_low, l_mid, r_mid, l_high, r_high = splitter(x, x)
    lo.append(l_low)
    mid.append(l_mid)
    hi.append(l_high)
lo=np.array(lo)
mid=np.array(mid)
hi=np.array(hi)

wavfile.write('lowband.wav', fs, np.int16(lo*32767))
wavfile.write('midband.wav', fs, np.int16(mid*32767))
wavfile.write('highband.wav', fs, np.int16(hi*32767))
reconstructed = lo + mid + hi
wavfile.write('reconstructed.wav', fs, np.int16(reconstructed*32767))
